<a href="https://colab.research.google.com/github/chandakrishabh/BERT/blob/master/Copy_of_01_how_to_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
# in this notebook we'll only get one of the files (the Oscar one) for the sake of simplicity and performance
!wget -c https://cdn-datasets.huggingface.co/EsperBERTo/data/oscar.eo.txt

--2020-06-06 12:00:37--  https://cdn-datasets.huggingface.co/EsperBERTo/data/oscar.eo.txt
Resolving cdn-datasets.huggingface.co (cdn-datasets.huggingface.co)... 13.224.166.87, 13.224.166.92, 13.224.166.71, ...
Connecting to cdn-datasets.huggingface.co (cdn-datasets.huggingface.co)|13.224.166.87|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 312733741 (298M) [text/plain]
Saving to: ‘oscar.eo.txt’

oscar.eo.txt        100%[===================>] 298.25M  27.0MB/s    in 11s     

2020-06-06 12:00:49 (27.2 MB/s) - ‘oscar.eo.txt’ saved [312733741/312733741]



## 2. Train a tokenizer

We choose to train a byte-level Byte-pair encoding tokenizer (the same as GPT-2), with the same special tokens as RoBERTa. Let’s arbitrarily pick its size to be 52,000.

We recommend training a byte-level BPE (rather than let’s say, a WordPiece tokenizer like BERT) because it will start building its vocabulary from an alphabet of single bytes, so all words will be decomposable into tokens (no more `<unk>` tokens!).


In [0]:

# Install `transformers` from master
!pip install git+https://github.com/huggingface/transformers
!pip list | grep -E 'transformers|tokenizers'
# transformers version at notebook update --- 2.9.1
# tokenizers version at notebook update --- 0.7.0

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-1v00nume
  Running command git clone -q https://github.com/huggingface/transformers /tmp/pip-req-build-1v00nume
     |████████████████████████████████| 3.8MB 3.5MB/s 
     |████████████████████████████████| 1.1MB 35.9MB/s 
     |████████████████████████████████| 890kB 40.6MB/s 
  Created wheel for transformers: filename=transformers-2.11.0-cp36-none-any.whl size=675468 sha256=2c8a6ec1f12f12b42aa2a7ae1f748aefc7527433f5d0897df84be5a5bc77eb6f
  Stored in directory: /tmp/pip-ephem-wheel-cache-ec4ahc4s/wheels/70/d3/52/b3fa4f8b8ef04167ac62e5bb2accb62ae764db2a378247490e
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=6ab0eccf1642f1f12ba6d6abdcd04530c9489640c1edca90d2d10b8f3a5a5d12
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built transformers sacremoses
tokenizers               0.7.0          
tr

In [0]:
%%time 
from pathlib import Path

from tokenizers import BertWordPieceTokenizer

# paths = [str(x) for x in Path(".").glob("**/*.txt")]

# Initialize a tokenizer
tokenizer = BertWordPieceTokenizer()

# Customize training
tokenizer.train(files="/content/drive/My Drive/BERT/oscar.eo.txt", vocab_size=52_000, min_frequency=2, special_tokens=[
    "[UNK]",
    "[CLS]",
    "[SEP]",
    "[MASK]",
])


Now let's save files to disk

In [0]:

tokenizer.save("drive/My Drive/BERT/","my-bert")
# !mv oscar.eo.txt "drive/My Drive/BERT/"

['drive/My Drive/BERT/my-bert-vocab.txt']

In [0]:
from tokenizers.implementations import BertWordPieceTokenizer
from tokenizers.processors import BertProcessing


tokenizer = BertWordPieceTokenizer("/content/drive/My Drive/BERT/tok/my-bert-vocab.txt"
)

In [0]:

tokenizer.enable_truncation(max_length=512)

## 3. Train a language model from scratch

**Update:** This section follows along the [`run_language_modeling.py`](https://github.com/huggingface/transformers/blob/master/examples/language-modeling/run_language_modeling.py) script, using our new [`Trainer`](https://github.com/huggingface/transformers/blob/master/src/transformers/trainer.py) directly. Feel free to pick the approach you like best.

> We’ll train a RoBERTa-like model, which is a BERT-like with a couple of changes (check the [documentation](https://huggingface.co/transformers/model_doc/roberta.html) for more details).

As the model is BERT-like, we’ll train it on a task of *Masked language modeling*, i.e. the predict how to fill arbitrary tokens that we randomly mask in the dataset. This is taken care of by the example script.


In [0]:
# Check that we have a GPU
!nvidia-smi

Sun Jun  7 06:51:40 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.82       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P0    29W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [0]:
# Check that PyTorch sees it
import torch
torch.cuda.is_available()
# torch.cuda.get_device_name(0) 

True

### We'll define the following config for the model

In [0]:
from transformers import BertConfig

config = BertConfig(type_vocab_size=1,vocab_size = 52000)


Now let's re-create our tokenizer in transformers

In [0]:
from transformers import BertTokenizerFast

tokenizer = BertTokenizerFast.from_pretrained("/content/drive/My Drive/BERT/tok/my-bert-vocab.txt")

Calling BertTokenizerFast.from_pretrained() with the path to a single file or url is deprecated


Finally let's initialize our model.

**Important:**

As we are training from scratch, we only initialize from a config, not from an existing pretrained model or checkpoint.

In [0]:
from transformers import BertForMaskedLM
model = BertForMaskedLM(config=config)

In [0]:
model.num_parameters()
# => 126 million parameters

126620704

### Now let's build our training Dataset

We'll build our dataset by applying our tokenizer to our text file.

Here, as we only have one text file, we don't even need to customize our `Dataset`. We'll just use the `LineByLineDataset` out-of-the-box.

In [0]:
%%time
from transformers import LineByLineTextDataset

dataset = LineByLineTextDataset(
    tokenizer=tokenizer,
    file_path="/content/drive/My Drive/BERT/oscar.eo.txt",
    block_size=128,
)


CPU times: user 4min 17s, sys: 7.06 s, total: 4min 24s
Wall time: 1min 36s


Like in the [`run_language_modeling.py`](https://github.com/huggingface/transformers/blob/master/examples/language-modeling/run_language_modeling.py) script, we need to define a data_collator.

This is just a small helper that will help us batch different samples of the dataset together into an object that PyTorch knows how to perform backprop on.

In [0]:
from transformers import DataCollatorForLanguageModeling,DefaultDataCollator

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

# data_collator = DataCollator(
#     tokenizer=tokenizer, mlm=True, mlm_probability=0.15
# )


### Finally, we are all set to initialize our Trainer

In [0]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="/content/drive/My Drive/BERT",
    overwrite_output_dir=True,
    num_train_epochs=1,
    per_device_train_batch_size=64,
    save_steps=10_000,
    save_total_limit=2,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
    prediction_loss_only=True,
)



### Start training

In [0]:
trainer.train()

{"loss": 7.310811567306518, "learning_rate": 4.8358287365379566e-05, "epoch": 0.03283425269240872, "step": 500}
{"loss": 6.7022975597381595, "learning_rate": 4.671657473075913e-05, "epoch": 0.06566850538481744, "step": 1000}
{"loss": 6.534313973426819, "learning_rate": 4.5074862096138694e-05, "epoch": 0.09850275807722617, "step": 1500}
{"loss": 6.43565500831604, "learning_rate": 4.343314946151826e-05, "epoch": 0.1313370107696349, "step": 2000}
{"loss": 6.340189016342163, "learning_rate": 4.179143682689782e-05, "epoch": 0.1641712634620436, "step": 2500}
{"loss": 6.279437676429748, "learning_rate": 4.0149724192277385e-05, "epoch": 0.19700551615445233, "step": 3000}
{"loss": 6.21033773803711, "learning_rate": 3.850801155765695e-05, "epoch": 0.22983976884686105, "step": 3500}
{"loss": 6.136436590194702, "learning_rate": 3.686629892303651e-05, "epoch": 0.2626740215392698, "step": 4000}
{"loss": 6.088840294837952, "learning_rate": 3.522458628841608e-05, "epoch": 0.29550827423167847, "step": 

/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:200: UserWarning: Please also save or load the state of the optimzer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)


{"loss": 4.8396025657653805, "learning_rate": 1.5524034672970843e-05, "epoch": 0.6895193065405831, "step": 10500}
{"loss": 4.803360108375549, "learning_rate": 1.3882322038350407e-05, "epoch": 0.7223535592329918, "step": 11000}
{"loss": 4.764783235549927, "learning_rate": 1.2240609403729971e-05, "epoch": 0.7551878119254006, "step": 11500}
{"loss": 4.74830855512619, "learning_rate": 1.0598896769109535e-05, "epoch": 0.7880220646178093, "step": 12000}
{"loss": 4.692740347862244, "learning_rate": 8.957184134489099e-06, "epoch": 0.820856317310218, "step": 12500}
{"loss": 4.678542919635773, "learning_rate": 7.315471499868663e-06, "epoch": 0.8536905700026267, "step": 13000}
{"loss": 4.668208144187927, "learning_rate": 5.673758865248227e-06, "epoch": 0.8865248226950354, "step": 13500}
{"loss": 4.6381971192359925, "learning_rate": 4.032046230627791e-06, "epoch": 0.9193590753874442, "step": 14000}
{"loss": 4.632666209220886, "learning_rate": 2.390333596007355e-06, "epoch": 0.9521933280798529, "st

TrainOutput(global_step=15228, training_loss=5.45466524735191)

#### 🎉 Save final model (+ tokenizer + config) to disk

In [0]:
trainer.save_model("./drive/My Drive/BERT/")

## 4. Check that the LM actually trained

Aside from looking at the training and eval losses going down, the easiest way to check whether our language model is learning anything interesting is via the `FillMaskPipeline`.

Pipelines are simple wrappers around tokenizers and models, and the 'fill-mask' one will let you input a sequence containing a masked token (here, `<mask>`) and return a list of the most probable filled sequences, with their probabilities.



In [0]:
from transformers import pipeline

fill_mask = pipeline(
    "fill-mask",
    model="./drive/My Drive/BERT/",
    tokenizer=tokenizer
)

In [0]:
# The sun <mask>.
# =>
# [{'score': 0.02119220793247223,
#   'sequence': '<s> La suno estas.</s>',
#   'token': 316},
#  {'score': 0.012403824366629124,
#   'sequence': '<s> La suno situas.</s>',
#   'token': 2340},
#  {'score': 0.011061107739806175,
#   'sequence': '<s> La suno estis.</s>',
#   'token': 394},
#  {'score': 0.008284995332360268,
#   'sequence': '<s> La suno de.</s>',
#   'token': 274},
#  {'score': 0.006471084896475077,
#   'sequence': '<s> La suno akvo.</s>',
#   'token': 1833}]

fill_mask("La suno [MASK].")

[{'score': 0.21564920246601105,
  'sequence': '[CLS] la suno.. [SEP]',
  'token': 17,
  'token_str': '.'},
 {'score': 0.006355357356369495,
  'sequence': '[CLS] la suno estas. [SEP]',
  'token': 1564,
  'token_str': 'estas'},
 {'score': 0.005034566391259432,
  'sequence': '[CLS] la suno —. [SEP]',
  'token': 411,
  'token_str': '—'},
 {'score': 0.004071058705449104,
  'sequence': '[CLS] la suno staris. [SEP]',
  'token': 4709,
  'token_str': 'staris'},
 {'score': 0.00404428830370307,
  'sequence': '[CLS] la suno,. [SEP]',
  'token': 15,
  'token_str': ','}]

Ok, simple syntax/grammar works. Let’s try a slightly more interesting prompt:



In [0]:
fill_mask("Jen la komenco de bela [MASK].")

# This is the beginning of a beautiful <mask>.
# # =>
# [{'score': 0.01814725436270237,
#   'sequence': '<s> Jen la komenco de bela urbo.</s>',
#   'token': 871},
#  {'score': 0.015888698399066925,
#   'sequence': '<s> Jen la komenco de bela vivo.</s>',
#   'token': 1160},
#  {'score': 0.015662025660276413,
#   'sequence': '<s> Jen la komenco de bela tempo.</s>',
#   'token': 1021},
#  {'score': 0.015555007383227348,
#   'sequence': '<s> Jen la komenco de bela mondo.</s>',
#   'token': 945},
#  {'score': 0.01412549614906311,
#   'sequence': '<s> Jen la komenco de bela tago.</s>',
#   'token': 1633}]

[{'score': 0.022457629442214966,
  'sequence': '[CLS] jen la komenco de bela tago. [SEP]',
  'token': 2591,
  'token_str': 'tago'},
 {'score': 0.018763139843940735,
  'sequence': '[CLS] jen la komenco de bela.. [SEP]',
  'token': 17,
  'token_str': '.'},
 {'score': 0.015563951805233955,
  'sequence': '[CLS] jen la komenco de bela urbo. [SEP]',
  'token': 2057,
  'token_str': 'urbo'},
 {'score': 0.0128762386739254,
  'sequence': '[CLS] jen la komenco de bela domo. [SEP]',
  'token': 2985,
  'token_str': 'domo'},
 {'score': 0.011341674253344536,
  'sequence': '[CLS] jen la komenco de bela vivo. [SEP]',
  'token': 2299,
  'token_str': 'vivo'}]

## 5. Share your model 🎉

Finally, when you have a nice model, please think about sharing it with the community:

- upload your model using the CLI: `transformers-cli upload`
- write a README.md model card and add it to the repository under `model_cards/`. Your model card should ideally include:
    - a model description,
    - training params (dataset, preprocessing, hyperparameters), 
    - evaluation results,
    - intended uses & limitations
    - whatever else is helpful! 🤓

### **TADA!**

➡️ Your model has a page on http://huggingface.co/models and everyone can load it using `AutoModel.from_pretrained("username/model_name")`.

[![tb](https://huggingface.co/blog/assets/01_how-to-train/model_page.png)](https://huggingface.co/julien-c/EsperBERTo-small)


If you want to take a look at models in different languages, check https://huggingface.co/models

[![all models](https://huggingface.co/front/thumbnails/models.png)](https://huggingface.co/models)
